In [1]:
import threading
import queue
import sounddevice as sd
import numpy as np
import soundfile as sf
import subprocess
from pathlib import Path
import cv2
import sounddevice as sd
from qwen_tts import Qwen3TTSModel
from datetime import datetime

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

from flash_attn import flash_attn_func  # или flash_attn_qkvpacked_func и т.д.
print("flash_attn установлен успешно")

True
NVIDIA GeForce RTX 4090
flash_attn установлен успешно


In [3]:
from src.asr.whisper_asr import WhisperTranscriber

In [4]:
from src.voice_clone.clone_manager import VoiceCloneManager
from qwen_tts import Qwen3TTSModel
import soundfile as sf

In [5]:
# Параметры записи
SAMPLE_RATE = 16000  # Гц
DURATION_MAX = 60    # макс. время записи в секундах
AUDIO_QUEUE = queue.Queue()

In [6]:
tts_model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-0.6B-Base",
    device_map="cuda:0",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# Callback для записи аудио в очередь
def audio_callback(indata, frames, time, status):
    if status:
        print(f"Запись: {status}")
    AUDIO_QUEUE.put(bytes(indata))


In [8]:
# Функция записи аудио в отдельном потоке
def record_audio(stop_event: threading.Event):
    with sd.RawInputStream(
        samplerate=16000,
        blocksize=1600,
        dtype='int16',
        channels=1,
        callback=audio_callback
    ):
        # Ждём пока не установлен флаг остановки
        while not stop_event.is_set():
            sd.sleep(100)  # проверяем флаг каждые 100 мс

In [9]:
# 1. Запуск потока записи
stop_event = threading.Event()
record_thread = threading.Thread(target=record_audio, args=(stop_event,))
record_thread.start()

input("Говори, нажми Enter для остановки\n")
stop_event.set()
record_thread.join()
print("Запись завершена.")

Говори, нажми Enter для остановки
 


Запись завершена.


In [10]:
# Собираем записанные блоки в один numpy-массив
audio_data_bytes = b"".join(list(AUDIO_QUEUE.queue))
if len(audio_data_bytes) == 0:
    print("Не было записано ни одного блока аудио.")
    exit()

In [11]:
# Конвертируем в numpy array float32 в диапазоне [-1,1]
audio_int16 = np.frombuffer(audio_data_bytes, dtype=np.int16)
audio_np = audio_int16.astype(np.float32) / 32768.0

In [12]:
# 2. Распознавание речи (Whisper)
transcriber = WhisperTranscriber()  # или другая модель Whisper
text, info = transcriber.transcribe(audio_np)

print(f"Распознанный текст: \"{text}\"")

Распознанный текст: " Ненавижу детей, ненавижу этих маленьких ёбаных чертей."


In [13]:
# 3. Генерация ответа (заглушка)
response_text = f"Ты сказал: {text}"
print(response_text)

Ты сказал:  Ненавижу детей, ненавижу этих маленьких ёбаных чертей.


In [14]:
# Инициализация менеджера
manager = VoiceCloneManager(model=tts_model)

# Имя человека для клонирования
person_name = "Julia"

# Получаем или создаем prompt
prompt_items = manager.load_or_create_clone(person_name)

clone_path: voices\Julia.pkl


In [15]:
# Генерация аудио
sentences = response_text
languages = "Russian"

wavs, sr = tts_model.generate_voice_clone(
    text=sentences,
    language=languages,
    voice_clone_prompt=prompt_items,
)

# Сохраняем первый результат
response_wav = (
    Path("E:/Coding/talking-head-assistant/generated_speech_audio")
    / f"{person_name}_clone.wav"
)

sf.write(response_wav, wavs[0], sr)
print(f"Аудио сохранено: {person_name}_clone.wav")


Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.


Аудио сохранено: Julia_clone.wav


In [16]:
WAV2LIP_PYTHON = r"C:\Users\Shurik\anaconda3\envs\wav2lip\python.exe"
WAV2LIP_DIR = Path(r"E:\Coding\talking-head-assistant\third_party\Wav2Lip")
RESULT_VIDEO_DIR = Path(r"E:\Coding\talking-head-assistant\result_video")

stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_video_path = RESULT_VIDEO_DIR / f"{person_name}_{stamp}.mp4"

In [17]:
wav2lip_cmd = [
    WAV2LIP_PYTHON,
    "inference.py",
    "--checkpoint_path", "checkpoints/wav2lip_gan.pth",
    "--face", "face_video.mp4",
    "--audio", str(response_wav),
    "--outfile", str(result_video_path),
    "--nosmooth"
]
print("Запускаем Wav2Lip для синхронизации губ...")
subprocess.run(
    wav2lip_cmd,
    cwd=WAV2LIP_DIR,
    check=True
)

Запускаем Wav2Lip для синхронизации губ...


CompletedProcess(args=['C:\\Users\\Shurik\\anaconda3\\envs\\wav2lip\\python.exe', 'inference.py', '--checkpoint_path', 'checkpoints/wav2lip_gan.pth', '--face', 'face_video.mp4', '--audio', 'E:\\Coding\\talking-head-assistant\\generated_speech_audio\\Julia_clone.wav', '--outfile', 'E:\\Coding\\talking-head-assistant\\result_video\\Julia_20260205_212825.mp4', '--nosmooth'], returncode=0)

In [18]:
# 6. Воспроизведение полученного видео через OpenCV
def play_audio(path):
    data, sr = sf.read(path)
    sd.play(data, sr)
    sd.wait()

audio_thread = threading.Thread(
    target=play_audio,
    args=(response_wav,),
    daemon=True
)
audio_thread.start()

cap = cv2.VideoCapture(str(result_video_path))
if not cap.isOpened():
    print("Ошибка: не удалось открыть видео result_video.mp4.")
print("Воспроизведение видео (нажмите 'q' для выхода)...")
while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Avatar", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Воспроизведение видео (нажмите 'q' для выхода)...
